<a href="https://colab.research.google.com/github/OscarJCC/Google-Colab-Professional-Internship-Project-Machine-Learning/blob/main/Div_Org_y_Ext_de_imag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow
print(tensorflow.__version__)

2.15.0


In [3]:
import cv2
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import json
import os
from tensorflow import keras
from glob import glob
from tqdm import notebook
from sklearn import preprocessing
from keras import Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras import optimizers
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import sys
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.models import model_from_json
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.optimizers import Adam

In [4]:
!pip install -U git+https://github.com/qubvel/efficientnet
from efficientnet.keras import EfficientNetB7
from efficientnet.keras import preprocess_input

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-6e5erwn5
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel/efficientnet /tmp/pip-req-build-6e5erwn5
  Resolved https://github.com/qubvel/efficientnet to commit f7f3e736c113b872caf53dae9fbbda996a8eb87d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 844.5 kB/s eta 0:00:00
  Created wheel for efficientnet: filename=efficientnet-1.1.1-py3-none-any.whl size=18427 sha256=7f486e644fe37def2010524113eb138444a30f50446478a9428452f1f113030d
  Stored in directory: /tmp/pip-ephem-wheel-cache-afwc01xf/wheels/bb/0f/38/154505705a0899276d49f521ee621a61bc2a8767026220f751
Successfully built efficientnet


In [5]:
train = '/content/drive/My Drive/KDEF_and_AKDEF_R/KDEF'
print(train)

/content/drive/My Drive/KDEF_and_AKDEF_R/KDEF


In [6]:
eff_net = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

258434480/258434480 [==============================] - 3s 0us/step


In [7]:
x = eff_net.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(7, activation='softmax')(x) # Colocar el numero de carpetas con imagenes a analizar
model = Model(inputs=eff_net.input, outputs=predictions)

# fix the feature extraction part of the model
# fix the feature extraction part of the model
for layer in eff_net.layers:
    layer.trainable = True

model.compile(optimizers.legacy.RMSprop(lr=0.0001, decay=0.0001/50),loss='categorical_crossentropy',metrics=['accuracy'])
#model.compile(optimizers.legacy.Adam(lr=0.0001, decay=0.0001/50),loss='categorical_crossentropy',metrics=['accuracy'])
#model.compile(optimizers.legacy.SGD(lr=0.0001, decay=0.0001/50),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 stem_conv (Conv2D)          (None, 112, 112, 64)         1728      ['input_1[0][0]']             
                                                                                                  
 stem_bn (BatchNormalizatio  (None, 112, 112, 64)         256       ['stem_conv[0][0]']           
 n)                                                                                               
                                                                                                  
 stem_activation (Activatio  (None, 112, 112, 64)         0         ['stem_bn[0][0]']         

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 block1b_project_conv (Conv  (None, 112, 112, 32)         1024      ['block1b_se_excite[0][0]']   
 2D)                                                                                              
                                                                                                  
 block1b_project_bn (BatchN  (None, 112, 112, 32)         128       ['block1b_project_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 block1b_drop (FixedDropout  (None, 112, 112, 32)         0         ['block1b_project_bn[0][0]']  
 )                                                                                                
                                                                                                  
 block1b_add (Add)           (None, 112, 112, 32)         0         ['block1b_drop[0][0]',        
          

In [8]:
class DataLoader():
  def __init__(self, path):
    self.path = path

  def preprocessing(self,instancePath):
    image = load_img(instancePath, target_size=(224, 224))
    image = img_to_array(image)
    image /= 255.0
    image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    return image

  def prepare_train_data(self):
    X_train, Y_train, Z_train = [], [], []

    for class_folder_name in os.listdir(self.path):
      class_folder_path = os.path.join(self.path, class_folder_name)
      for img in notebook.tqdm(os.listdir(class_folder_path)):
        imgpath = os.path.join(class_folder_path, img)
        img = self.preprocessing(imgpath)
        X_train.append(img)
        Y_train.append(class_folder_name)
        Z_train.append(imgpath[-11:-4])

    X_train, Y_train, Z_train = map(np.asarray, [X_train, Y_train, Z_train])

    le = preprocessing.LabelEncoder()
    Y_train = le.fit_transform(Y_train)
    print(np.unique(Y_train))

    print(f'X_train shape:{X_train.shape}\nY_train shape:{Y_train.shape}\nZ_train shape:{Z_train.shape}')
    return X_train,Y_train,Z_train

Funcion -> Eliminacion De Personas

In [9]:
def eP(pe,lx,ly,lz):
  i = np.where(lz == pe)[0]

  people_x_train_pe = np.delete(lx, i, axis=0)
  people_y_train_pe = np.delete(ly, i, axis=0)
  people_z_train_pe = np.delete(lz, i, axis=0)

  return people_x_train_pe, people_y_train_pe, people_z_train_pe, i

In [11]:
X_train, Y_train, Z_train = [], [], []

dl = DataLoader(path = train)
X_train, Y_train, Z_train = dl.prepare_train_data()

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

[0 1 2 3 4 5 6]
X_train shape:(490, 224, 224, 3)
Y_train shape:(490,)
Z_train shape:(490,)


In [12]:
Y_train_ohe = to_categorical(Y_train, 7) # Colocar el numero de carpetas con imagenes a analizar
print(f'X_train shape:{X_train.shape}\nY_train shape:{Y_train_ohe.shape}')

X_train shape:(490, 224, 224, 3)
Y_train shape:(490, 7)


In [13]:
indices = np.arange(X_train.shape[0]/7)

people_Z_train = [0]*int(X_train.shape[0]/7)

for i in range(len(indices)):
  people_Z_train[i] = Z_train[i][0:4]

people_Z_train = np.asarray(people_Z_train)

print("People:\n",people_Z_train,people_Z_train.shape[0])

People:
 ['AF01' 'AF02' 'AF04' 'AF03' 'AF05' 'AF06' 'AF07' 'AF08' 'AF10' 'AF09'
 'AF12' 'AF11' 'AF14' 'AF13' 'AF16' 'AF15' 'AF17' 'AF19' 'AF18' 'AF20'
 'AF21' 'AF22' 'AF23' 'AF24' 'AF25' 'AF27' 'AF26' 'AF29' 'AF28' 'AF30'
 'AF31' 'AF32' 'AF33' 'AF34' 'AF35' 'AM01' 'AM02' 'AM04' 'AM03' 'AM05'
 'AM06' 'AM07' 'AM08' 'AM09' 'AM10' 'AM11' 'AM12' 'AM14' 'AM13' 'AM15'
 'AM16' 'AM17' 'AM19' 'AM18' 'AM20' 'AM21' 'AM22' 'AM23' 'AM24' 'AM25'
 'AM26' 'AM27' 'AM28' 'AM29' 'AM30' 'AM31' 'AM32' 'AM33' 'AM34' 'AM35'] 70


In [14]:
emotion, y_emotion = [], []
for i in range(len(people_Z_train)):
  print("\nPerson",people_Z_train[i],":")
  for j in range(len(Z_train)):
    if people_Z_train[i] == Z_train[j][0:4]:
      print(Z_train[j],"->",Y_train_ohe[j])
      if len(emotion) < 7:
        emotion.append(Z_train[j])
        y_emotion.append(Y_train_ohe[j])
emotion, y_emotion = map(np.asarray, [emotion, y_emotion])


Person AF01 :
AF01DIS -> [0. 1. 0. 0. 0. 0. 0.]
AF01AFS -> [0. 0. 1. 0. 0. 0. 0.]
AF01SAS -> [0. 0. 0. 0. 0. 1. 0.]
AF01NES -> [0. 0. 0. 0. 1. 0. 0.]
AF01SUS -> [0. 0. 0. 0. 0. 0. 1.]
AF01ANS -> [1. 0. 0. 0. 0. 0. 0.]
AF01HAS -> [0. 0. 0. 1. 0. 0. 0.]

Person AF02 :
AF02DIS -> [0. 1. 0. 0. 0. 0. 0.]
AF02AFS -> [0. 0. 1. 0. 0. 0. 0.]
AF02SAS -> [0. 0. 0. 0. 0. 1. 0.]
AF02NES -> [0. 0. 0. 0. 1. 0. 0.]
AF02SUS -> [0. 0. 0. 0. 0. 0. 1.]
AF02ANS -> [1. 0. 0. 0. 0. 0. 0.]
AF02HAS -> [0. 0. 0. 1. 0. 0. 0.]

Person AF04 :
AF04DIS -> [0. 1. 0. 0. 0. 0. 0.]
AF04AFS -> [0. 0. 1. 0. 0. 0. 0.]
AF04SAS -> [0. 0. 0. 0. 0. 1. 0.]
AF04NES -> [0. 0. 0. 0. 1. 0. 0.]
AF04SUS -> [0. 0. 0. 0. 0. 0. 1.]
AF04ANS -> [1. 0. 0. 0. 0. 0. 0.]
AF04HAS -> [0. 0. 0. 1. 0. 0. 0.]

Person AF03 :
AF03DIS -> [0. 1. 0. 0. 0. 0. 0.]
AF03AFS -> [0. 0. 1. 0. 0. 0. 0.]
AF03SAS -> [0. 0. 0. 0. 0. 1. 0.]
AF03NES -> [0. 0. 0. 0. 1. 0. 0.]
AF03SUS -> [0. 0. 0. 0. 0. 0. 1.]
AF03ANS -> [1. 0. 0. 0. 0. 0. 0.]
AF03HAS -> [0. 0. 0. 1

Union y organizacion de las imagenes de cada persona asociandolas a un indice

In [15]:
people_X_train = [0]*people_Z_train.shape[0]
people_Y_train_ohe = [0]*people_Z_train.shape[0]

for i in range(len(people_Z_train)):
  k = 0
  people_x = [0]*7
  people_y = [0]*7
  for j in range(len(Z_train)):

    if people_Z_train[i] == Z_train[j][0:4]:
      people_x[k] = X_train[j]
      people_y[k] = Y_train_ohe[j]
      k += 1
  people_X_train[i] = people_x
  people_Y_train_ohe[i] = people_y

people_X_train, people_Y_train_ohe, people_Z_train = map(np.array, [people_X_train, people_Y_train_ohe, people_Z_train])

print(f'People_X_train shape:{people_X_train.shape}\nPeople_Y_train shape:{people_Y_train_ohe.shape}')

People_X_train shape:(70, 7, 224, 224, 3)
People_Y_train shape:(70, 7, 7)


Division de las personas en train 90% y valid 10%

In [16]:
people_x_train, people_x_valid, people_y_train, people_y_valid, indices_train, indices_valid = train_test_split(people_X_train, people_Y_train_ohe, indices, test_size=0.1, random_state=42)

print(f'People_X_train shape:{people_x_train.shape}\nPeople_Y_train shape:{people_y_train.shape}\n')
print(f'People_X_valid shape:{people_x_valid.shape}\nPeople_Y_valid shape:{people_y_valid.shape}\n')

People_X_train shape:(63, 7, 224, 224, 3)
People_Y_train shape:(63, 7, 7)

People_X_valid shape:(7, 7, 224, 224, 3)
People_Y_valid shape:(7, 7, 7)



Muestra personas en train y en valid

In [17]:
people_z_train = people_Z_train[np.array(indices_train, dtype=int)]
people_z_valid = people_Z_train[np.array(indices_valid, dtype=int)]

print("People in train:\n",people_z_train,people_z_train.shape[0])
print("People in valid:\n",people_z_valid,people_z_valid.shape[0])

People in train:
 ['AF34' 'AM11' 'AF14' 'AF32' 'AF09' 'AM33' 'AF06' 'AM28' 'AF31' 'AM24'
 'AM01' 'AM23' 'AM14' 'AF17' 'AF35' 'AM08' 'AF28' 'AF08' 'AM18' 'AM06'
 'AM10' 'AM12' 'AF20' 'AM22' 'AM05' 'AF27' 'AM03' 'AF13' 'AM16' 'AF03'
 'AF19' 'AF10' 'AM21' 'AF07' 'AM02' 'AM30' 'AM35' 'AM34' 'AF15' 'AF29'
 'AM07' 'AF26' 'AM13' 'AF25' 'AM25' 'AM29' 'AF11' 'AF33' 'AM32' 'AM27'
 'AM04' 'AF30' 'AM09' 'AM31' 'AF02' 'AM19' 'AF22' 'AF04' 'AF24' 'AF21'
 'AM26' 'AF16' 'AM17'] 63
People in valid:
 ['AF23' 'AF01' 'AM15' 'AF05' 'AM20' 'AF18' 'AF12'] 7


In [18]:
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_x_train.npy', people_x_train)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_x_valid.npy', people_x_valid)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_y_train.npy', people_y_train)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_y_valid.npy', people_y_valid)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_z_train.npy', people_z_train)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/Emocion.npy', emotion)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/y_Emocion.npy', y_emotion)

Eliminacion de solo la personas 1


In [19]:
pe1 = people_z_train[0] #Persona eliminada 1

people_x_train_pe1, people_y_train_pe1, people_z_train_pe1, i1 = eP(pe1,people_x_train,people_y_train,people_z_train)

print(f'People_X_train shape:{people_x_train_pe1.shape}\nPeople_Y_train shape:{people_y_train_pe1.shape}\n')

People_X_train shape:(62, 7, 224, 224, 3)
People_Y_train shape:(62, 7, 7)



In [20]:
print("People in train:\n",people_z_train_pe1,people_z_train_pe1.shape[0])
print("People in valid:\n",people_z_valid,people_z_valid.shape[0])

People in train:
 ['AM11' 'AF14' 'AF32' 'AF09' 'AM33' 'AF06' 'AM28' 'AF31' 'AM24' 'AM01'
 'AM23' 'AM14' 'AF17' 'AF35' 'AM08' 'AF28' 'AF08' 'AM18' 'AM06' 'AM10'
 'AM12' 'AF20' 'AM22' 'AM05' 'AF27' 'AM03' 'AF13' 'AM16' 'AF03' 'AF19'
 'AF10' 'AM21' 'AF07' 'AM02' 'AM30' 'AM35' 'AM34' 'AF15' 'AF29' 'AM07'
 'AF26' 'AM13' 'AF25' 'AM25' 'AM29' 'AF11' 'AF33' 'AM32' 'AM27' 'AM04'
 'AF30' 'AM09' 'AM31' 'AF02' 'AM19' 'AF22' 'AF04' 'AF24' 'AF21' 'AM26'
 'AF16' 'AM17'] 62
People in valid:
 ['AF23' 'AF01' 'AM15' 'AF05' 'AM20' 'AF18' 'AF12'] 7


In [21]:
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_x_train_pe1.npy', people_x_train_pe1)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_y_train_pe1.npy', people_y_train_pe1)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/x_valid_pe1.npy', people_x_train[i1])

Eliminacion de solo la personas 2

In [22]:
pe2 = people_z_train[1] #Persona eliminada 2

people_x_train_pe2, people_y_train_pe2, people_z_train_pe2, i2 = eP(pe2,people_x_train,people_y_train,people_z_train)

print(f'People_X_train shape:{people_x_train_pe2.shape}\nPeople_Y_train shape:{people_y_train_pe2.shape}\n')

People_X_train shape:(62, 7, 224, 224, 3)
People_Y_train shape:(62, 7, 7)



In [23]:
print("People in train:\n",people_z_train_pe2,people_z_train_pe2.shape[0])
print("People in valid:\n",people_z_valid,people_z_valid.shape[0])

People in train:
 ['AF34' 'AF14' 'AF32' 'AF09' 'AM33' 'AF06' 'AM28' 'AF31' 'AM24' 'AM01'
 'AM23' 'AM14' 'AF17' 'AF35' 'AM08' 'AF28' 'AF08' 'AM18' 'AM06' 'AM10'
 'AM12' 'AF20' 'AM22' 'AM05' 'AF27' 'AM03' 'AF13' 'AM16' 'AF03' 'AF19'
 'AF10' 'AM21' 'AF07' 'AM02' 'AM30' 'AM35' 'AM34' 'AF15' 'AF29' 'AM07'
 'AF26' 'AM13' 'AF25' 'AM25' 'AM29' 'AF11' 'AF33' 'AM32' 'AM27' 'AM04'
 'AF30' 'AM09' 'AM31' 'AF02' 'AM19' 'AF22' 'AF04' 'AF24' 'AF21' 'AM26'
 'AF16' 'AM17'] 62
People in valid:
 ['AF23' 'AF01' 'AM15' 'AF05' 'AM20' 'AF18' 'AF12'] 7


In [24]:
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_x_train_pe2.npy', people_x_train_pe2)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_y_train_pe2.npy', people_y_train_pe2)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/x_valid_pe2.npy', people_x_train[i2])

Eliminacion de la personas 1 y 2

In [25]:
people_x_train_pe12, people_y_train_pe12, people_z_train_pe12,i = eP(pe1,people_x_train,people_y_train,people_z_train)
people_x_train_pe12, people_y_train_pe12, people_z_train_pe12,i = eP(pe2,people_x_train_pe12,people_y_train_pe12,people_z_train_pe12)

print(f'People_X_train shape:{people_x_train_pe12.shape}\nPeople_Y_train shape:{people_y_train_pe12.shape}\n')

People_X_train shape:(61, 7, 224, 224, 3)
People_Y_train shape:(61, 7, 7)



In [26]:
print("People in train:\n",people_z_train_pe12,people_z_train_pe12.shape[0])
print("People in valid:\n",people_z_valid,people_z_valid.shape[0])

People in train:
 ['AF14' 'AF32' 'AF09' 'AM33' 'AF06' 'AM28' 'AF31' 'AM24' 'AM01' 'AM23'
 'AM14' 'AF17' 'AF35' 'AM08' 'AF28' 'AF08' 'AM18' 'AM06' 'AM10' 'AM12'
 'AF20' 'AM22' 'AM05' 'AF27' 'AM03' 'AF13' 'AM16' 'AF03' 'AF19' 'AF10'
 'AM21' 'AF07' 'AM02' 'AM30' 'AM35' 'AM34' 'AF15' 'AF29' 'AM07' 'AF26'
 'AM13' 'AF25' 'AM25' 'AM29' 'AF11' 'AF33' 'AM32' 'AM27' 'AM04' 'AF30'
 'AM09' 'AM31' 'AF02' 'AM19' 'AF22' 'AF04' 'AF24' 'AF21' 'AM26' 'AF16'
 'AM17'] 61
People in valid:
 ['AF23' 'AF01' 'AM15' 'AF05' 'AM20' 'AF18' 'AF12'] 7


In [27]:
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_x_train_pe12.npy', people_x_train_pe12)
np.save('/content/drive/My Drive/KDEF_and_AKDEF_R/NrmEfficientNet/People_Data/People_y_train_pe12.npy', people_y_train_pe12)